In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install -e transformers
!pip install fugashi[unidic-lite]
!pip install ipadic

In [ ]:
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

In [ ]:
# 依存パッケージのインストール
!pip install sentencepiece==0.1.91

In [ ]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

# テキストの準備
text = 'こんにちは、'

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained('rinna/japanese-gpt2-medium')
model = AutoModelForCausalLM.from_pretrained('rinna/japanese-gpt2-medium')

# テキストをテンソルに変換
input = tokenizer.encode(text, return_tensors='pt')

# 推論
model.eval()
with torch.no_grad():
    output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=1)
    print(tokenizer.batch_decode(output))

In [ ]:
#「transformers/examples/language-modeling/run_clm.py」の編集
#・T5Tokenizerのインポートの追加 (from transformers import T5Tokenizer)
#・AutoTokenizerをT5Tokenizerに変更・AutoTokenizerをT5Tokenizerに変更

In [ ]:
!ls ./transformers/examples/language-modeling/

In [ ]:
%%time
# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --do_train \
    --do_eval \
    --train_file=train.txt \
    --validation_file=train.txt \
    --num_train_epochs=100 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/ \
    --use_fast_tokenizer=False \
    --logging_steps=50

In [ ]:
# 学習状況の確認
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

# テキストの準備
text = 'こんにちは、'

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained('rinna/japanese-gpt2-medium')
model = AutoModelForCausalLM.from_pretrained('output/')

# テキストをテンソルに変換
input = tokenizer.encode(text, return_tensors='pt')

# 推論
model.eval()
with torch.no_grad():
    output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=1)
    print(tokenizer.batch_decode(output))